# Module 1 Practice 1 Answers - Working with trial structure in the SDTM
In this practice exercise, you will read the SDTM and join datasets together.  You will need to reference the documentation to find the proper keys for doing this.  This will give you some practice in navigating the SDTM documentation.

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade "pandas>=1.1"
!{sys.executable} -m pip install xmltodict

import pandas as pd
import numpy as np


# Find information about the trial arms in the study

We would like to know how many trial arms there are in this study.  Find and print the unique trial arms.

We reference the [SDTM Implementation Guide](../resources/SDTMIG_v3.3_FINAL.pdf), section 3, page 19.  We find that the trial arms are stored in the `ta` dataset.

In [ ]:
with open('../resources/SDTM_sample/ta.xpt', 'rb') as f:
    ta = pd.read_sas(f, format='xport', encoding='utf-8')
    
display(ta)
display(ta['ARM'].unique())

There were three unique trial arms:
  * Placebo
  * Xanomeline High Dose
  * Xanomeline Low Dose

## How long was each trial arm
We would like to know how long each trial arm was planned to last.  This will be a sum of the duration planned for each element of the trial arm.  You will note that each arm has different elements in it.  The high dose arm has more elements that the placebo and low dose arms, so each arm must be summed up individually as it's possible the arms were planned to have different lengths.  

We have to look the duration of each element up in another table.  Examine the [SDTM Implementation Guide](resources/SDTMIG_v3.3_FINAL.pdf) and look for which table has information on the elements.  You can search for the table that has the ETCD key in it, for example.

In the SDTMIG on page 20, we find that the `TE` (Trial Elements) dataset has the ETCD in it.  The [SDTM](resources/SDTM_v1.8.pdf) (pages 27-28) tells us that the Trial Elements dataset contains the `TEDUR` field, which is the planned duration of the trial element.  It further gives us the format of the field as ISO 8601.  We can search the web for a description of the ISO standard, and find the format to decipher the data (i.e. wikipedia links to an archived version of the standard on the Library of Congress, https://web.archive.org/web/20171019211402/https://www.loc.gov/standards/datetime/ISO_DIS%208601-1.pdf section 4.4.3.2).

### Open the proper data set
Using the dataset you discovered from the SDTMIG, read the correct datafile into a Pandas dataframe.

In [ ]:
with open('../resources/SDTM_sample/te.xpt', 'rb') as f:
    te = pd.read_sas(f, format='xport', encoding='utf-8')
    
display(te)

### Join the datasets together on the proper key

ETCD from Trial Elements joins to ETCD in Trial Arms.  This will link the duration of each Trial Element to the Trial Arm that it was in.

In [ ]:
trial_arm_elements = te.join(ta.set_index('ETCD'), how='left', on='ETCD', lsuffix='TE_')

### Sum the duration for each arm
The durations will be in ISO 8601 format, which you can find described online.  ISO 8601 describes a format for representing `durations`, which you will need to convert to an interger value to allow for summation.

We have to convert the ISO duration standard to something that we can sum.

ISO 8601 represents durations with the following format:
  * `P` - Identifies a Period specification
  * an integer followed by `W` identifies a duration in weeks.

All of the trial elements are given in duration of weeks.  A simple approach that works in this case is to strip non numeric characters from the duration, leaving us integers that are the duration in weeks which we can then sum.

In [ ]:
# strip non numeric characters
trial_arm_elements['TEDUR'] = trial_arm_elements['TEDUR'].str.replace(r"[^0-9]",'',regex=True)
# set all blanks to zero
trial_arm_elements['TEDUR'] = trial_arm_elements['TEDUR'].replace('', '0', regex=False)

# convert to int
trial_arm_elements = trial_arm_elements.astype({'TEDUR': int})

# finally groupby and sum
trial_arm_elements.groupby(by=['ARM'])['TEDUR'].sum()

Each arm was planned to last 26 weeks.